In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import (StructType, StructField, StringType,
                               DoubleType, IntegerType, LongType)
from pyspark.sql.functions import *
from pyspark import SparkConf
from pyspark import SparkContext
import multiprocessing
from pyspark.ml import Pipeline
import sys

print('Inicio del Script\n')

# Configuracion de memoria y cores
cores = multiprocessing.cpu_count()
p = 5
particiones = cores * p
conf = SparkConf()
conf.set("spark.sql.shuffle.partitions", particiones)
conf.set("spark.default.parallelism", particiones)
conf.set("spark.driver.cores", cores)
conf.set("spark.executor.cores", cores)
sc = SparkContext(conf=conf)


spark = SparkSession.builder.appName("Microsoft_Kaggle").getOrCreate()

Inicio del Script



In [2]:
df_num = spark.read.csv('../data/df_num/*.csv', header=True, inferSchema=True)

In [3]:
df_num.show(3)

+------+----------------+----------------+-------------------------+-------------------------+-------------------+-----------------+------+-----------------+--------------+----------------------+-----------------+---------------------------+-------+-------+-----------+---------------+-----+---------------+--------+------------+------------------------+-------------------------+-------------------------+--------------------------------------+-------------------------------+-------------------------------+--------------------------------+--------------------------+-----------------------+-------------------------------------------------+-------------------------------------------------+-----------------------------------------------+-------------------------------------+--------------------+----------------------+----------------------------------+---------------------------+--------------------------------+--------------------------+------------------------+---------------------+-------

In [97]:
len_print=55
null_list = list()
for i, j in zip(nulls[0], df_num.columns):
    if i != 0:
        len_a = len(j)
        dif = (len_print - len_a)*"."
        print(f"Columna\033[94m {j}\x1b[0m {dif} \033[93m{i} \x1b[0mnulls")
        null_list.append(j)

Columna RtpStateBitfield ....................................... 64540 nulls
Columna DefaultBrowsersIdentifier .............................. 16034179 nulls
Columna AVProductStatesIdentifier .............................. 59988 nulls
Columna AVProductsInstalled .................................... 59988 nulls
Columna AVProductsEnabled ...................................... 59988 nulls
Columna CityIdentifier ......................................... 517371 nulls
Columna OrganizationIdentifier ................................. 5233647 nulls
Columna GeoNameIdentifier ...................................... 360 nulls
Columna IsProtected ............................................ 59693 nulls
Columna SMode .................................................. 6369031 nulls
Columna IeVerIdentifier ........................................ 108690 nulls
Columna Firewall ............................................... 149822 nulls
Columna UacLuaenable ........................................... 187

In [106]:
df_num.columns

['IsBeta',
 'RtpStateBitfield',
 'IsSxsPassiveMode',
 'DefaultBrowsersIdentifier',
 'AVProductStatesIdentifier',
 'AVProductsInstalled',
 'AVProductsEnabled',
 'HasTpm',
 'CountryIdentifier',
 'CityIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'LocaleEnglishNameIdentifier',
 'OsBuild',
 'OsSuite',
 'IsProtected',
 'AutoSampleOptIn',
 'SMode',
 'IeVerIdentifier',
 'Firewall',
 'UacLuaenable',
 'Census_OEMNameIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorCoreCount',
 'Census_ProcessorManufacturerIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_PrimaryDiskTotalCapacity',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalRAM',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSInstallLanguageIdentifi

In [98]:
for c in null_list:
    df_num.select(c).describe().show()

+-------+------------------+
|summary|  RtpStateBitfield|
+-------+------------------+
|  count|          16710196|
|   mean| 6.848561082108193|
| stddev|1.0183299830475636|
|    min|                 0|
|    max|                40|
+-------+------------------+

+-------+-------------------------+
|summary|DefaultBrowsersIdentifier|
+-------+-------------------------+
|  count|                   740557|
|   mean|       1635.8635891632919|
| stddev|       1020.4608645386592|
|    min|                        1|
|    max|                     3214|
+-------+-------------------------+

+-------+-------------------------+
|summary|AVProductStatesIdentifier|
+-------+-------------------------+
|  count|                 16714748|
|   mean|        48594.02434275408|
| stddev|       13258.519103745984|
|    min|                        2|
|    max|                    70507|
+-------+-------------------------+

+-------+-------------------+
|summary|AVProductsInstalled|
+-------+-------------------

+-------+----------------------------------+
|summary|Census_OSInstallLanguageIdentifier|
+-------+----------------------------------+
|  count|                          16655909|
|   mean|                14.875570405674047|
| stddev|                  10.3658258530923|
|    min|                                 1|
|    max|                                39|
+-------+----------------------------------+

+-------+--------------------------+
|summary|Census_IsFlightingInternal|
+-------+--------------------------+
|  count|                   2692015|
|   mean|      1.262994448396461...|
| stddev|      0.003553841535583...|
|    min|                         0|
|    max|                         1|
+-------+--------------------------+

+-------+------------------------+
|summary|Census_IsFlightsDisabled|
+-------+------------------------+
|  count|                16488412|
|   mean|    7.641730446813192E-6|
| stddev|    0.002764357522866056|
|    min|                       0|
|    max|      

In [10]:
nulls = df_num.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_num.columns]).collect()

In [42]:
df_num.persist()
df_num.count()

16774736

In [74]:
GeoNameIdentifier = df_num.select('GeoNameIdentifier').distinct().collect()

In [75]:
GeoNameIdentifier

[Row(GeoNameIdentifier=148),
 Row(GeoNameIdentifier=243),
 Row(GeoNameIdentifier=165),
 Row(GeoNameIdentifier=105),
 Row(GeoNameIdentifier=146),
 Row(GeoNameIdentifier=10),
 Row(GeoNameIdentifier=77),
 Row(GeoNameIdentifier=31),
 Row(GeoNameIdentifier=58),
 Row(GeoNameIdentifier=102),
 Row(GeoNameIdentifier=None),
 Row(GeoNameIdentifier=188),
 Row(GeoNameIdentifier=259),
 Row(GeoNameIdentifier=204),
 Row(GeoNameIdentifier=251),
 Row(GeoNameIdentifier=206),
 Row(GeoNameIdentifier=207),
 Row(GeoNameIdentifier=11),
 Row(GeoNameIdentifier=1),
 Row(GeoNameIdentifier=33),
 Row(GeoNameIdentifier=85),
 Row(GeoNameIdentifier=179),
 Row(GeoNameIdentifier=137),
 Row(GeoNameIdentifier=189),
 Row(GeoNameIdentifier=150),
 Row(GeoNameIdentifier=52),
 Row(GeoNameIdentifier=50),
 Row(GeoNameIdentifier=110),
 Row(GeoNameIdentifier=65),
 Row(GeoNameIdentifier=37),
 Row(GeoNameIdentifier=83),
 Row(GeoNameIdentifier=255),
 Row(GeoNameIdentifier=267),
 Row(GeoNameIdentifier=53),
 Row(GeoNameIdentifier=68),


In [77]:
df_num.filter(col('GeoNameIdentifier').isNull()).show()

+------+----------------+----------------+-------------------------+-------------------------+-------------------+-----------------+------+-----------------+--------------+----------------------+-----------------+---------------------------+-------+-------+-----------+---------------+-----+---------------+--------+------------+------------------------+-------------------------+-------------------------+--------------------------------------+-------------------------------+-------------------------------+--------------------------------+--------------------------+-----------------------+-------------------------------------------------+-------------------------------------------------+-----------------------------------------------+-------------------------------------+--------------------+----------------------+----------------------------------+---------------------------+--------------------------------+--------------------------+------------------------+---------------------+-------

In [ ]:
df_num

In [80]:
CityIdentifier = df_num.select('CityIdentifier').distinct().collect()

In [103]:
df_num.groupBy('CityIdentifier').agg(count('*').alias('count')).sort(col("count").desc()).show()

+--------------+------+
|CityIdentifier| count|
+--------------+------+
|          null|517371|
|        130775|162642|
|         82373|152761|
|         16668|145213|
|         61668|135874|
|         10222|135508|
|         66202|128981|
|         58607|121349|
|         22656|121006|
|        143782|116040|
|         66953|114883|
|         92213|105453|
|         96401|104797|
|        136271| 82026|
|         37376| 79745|
|         74924| 76998|
|        143155| 71545|
|         39800| 67147|
|        159247| 65971|
|         75528| 65266|
+--------------+------+
only showing top 20 rows



In [105]:
df_num.select('CityIdentifier').agg(max('CityIdentifier').alias('max')).show()

+------+
|   max|
+------+
|167962|
+------+



In [107]:
df_ci = df_num.filter(col('CityIdentifier').isNull()).select('CityIdentifier', 'CountryIdentifier')

In [111]:
df_ci.groupBy('CountryIdentifier').agg(count('*').alias('count')).sort(col("count").desc()).show()

+-----------------+-----+
|CountryIdentifier|count|
+-----------------+-----+
|               95|59536|
|              107|37720|
|               44|30413|
|              155|25731|
|              158|16394|
|               11|12726|
|              141|12152|
|              160|11294|
|               43|10452|
|              203| 9752|
|               66| 8811|
|              201| 8544|
|                9| 8286|
|              214| 7937|
|              101| 7309|
|              171| 7247|
|               60| 6479|
|               86| 6362|
|              195| 6220|
|              151| 5848|
+-----------------+-----+
only showing top 20 rows

